# Creating an Automated Model Fine-Tuning Workflow with SageMaker Pipelines

# NOTE:  THIS NOTEBOOK WILL TAKE ABOUT 30 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

# SageMaker Pipelines

Amazon SageMaker Pipelines support the following:

* **Pipelines** - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* **Processing Job Steps** - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* **Training Job Steps** - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* **Conditional Steps** - Provides conditional execution of branches in a pipeline.
* **Registering Models** - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* **Parameterized Executions** - Allows pipeline executions to vary by supplied parameters.
* **Transform Job Steps** - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

# Our Pipeline

In the Processing Step, we perform Feature Engineering to tokenizer our dialogue inputs using the `transformer` library from HuggingFace/

In the Training Step, we fine-tune the model to summarize dialogue effectively on the `diagsum` dataset.

In the Evaluation Step, we take the fine-tuned model and a test dataset as input, and produce a JSON file containing evaluation metrics based on the ROUGE metric for summarization.

In the Condition Step, we decide whether to register this model if the metrics of the model, as determined by our evaluation step, exceeded some value. 

In [2]:
%pip install sagemaker==2.128.0
%pip install sagemaker-experiments==0.1.45

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 6.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00:00:01
  Created wheel for sagemaker: filename=sagemaker-2.128.0-py2.py3-none-any.whl size=896984 sha256=e61900a978d5fd9bda447ce4cac96ec0e8da1417bb518c5d33b93498a7307cb1
  Stored in directory: /root/.cache/pip/wheels/42/c5/a2/a56f01e627504ee78fce360e9d1cf74c415787526d78a2e692
  Created wheel for protobuf3-to-dict: filename=protobuf3_to_dict-0.1.5-py3-none-any.whl size=4010 sha256=7309e4ba8ad62d2d4fa2c06e6ea226e59e5cbec128186f928d96fb2c9e180f2a
  Stored in directory: /root/.cache/pip/wheels/12/3e/42/e783cdd4e7b8fda9bfc472eeb465bc9041bda90a3dbece8d74
Successfully built sagemaker protobuf3-to-dict
  Attempting uninstall: protobuf
    Found existing installation:

In [3]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

import botocore.config

config = botocore.config.Config(
    user_agent_extra='gaia/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)
s3 = boto3.Session().client(service_name="s3", 
                            region_name=region,
                            config=config)

# Set S3 Source Location

In [4]:
%store -r raw_input_data_s3_uri

In [5]:
try:
    raw_input_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not have the required datasets.       ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-550703975190/data-summarization/


In [7]:
if not raw_input_data_s3_uri:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not have the required datasets.       ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Track the Pipeline as an `Experiment`

In [8]:
import time

In [9]:
running_executions = 0
completed_executions = 0

try:
    existing_pipeline_executions_response = sm.list_pipeline_executions(
        PipelineName=pipeline_name,
        SortOrder="Descending",
    )

    if "PipelineExecutionSummaries" in existing_pipeline_executions_response.keys():
        if len(existing_pipeline_executions_response["PipelineExecutionSummaries"]) > 0:
            execution = existing_pipeline_executions_response["PipelineExecutionSummaries"][0]
            if "PipelineExecutionStatus" in execution:
                if execution["PipelineExecutionStatus"] == "Executing":
                    running_executions = running_executions + 1
                else:
                    completed_executions = completed_executions + 1

            print(
                "[INFO] You have {} Pipeline execution(s) currently running and {} execution(s) completed.".format(
                    running_executions, completed_executions
                )
            )
    else:
        print("[OK] Please continue.")
except:
    pass

if running_executions == 0:
    timestamp = int(time.time())
    pipeline_name = "dialogue-summary-pipeline-{}".format(timestamp)
    print("Created Pipeline Name: " + pipeline_name)

Created Pipeline Name: dialogue-summary-pipeline-1720217224


In [10]:
print(pipeline_name)

dialogue-summary-pipeline-1720217224


In [11]:
%store pipeline_name

Stored 'pipeline_name' (str)


In [12]:
from smexperiments.experiment import Experiment

pipeline_experiment = Experiment.create(
    experiment_name=pipeline_name,
    description="Dialogue Summarization Pipeline Experiment",
    sagemaker_boto_client=sm,
)
pipeline_experiment_name = pipeline_experiment.experiment_name
print("Created Pipeline Experiment Name: {}".format(pipeline_experiment_name))

Created Pipeline Experiment Name: dialogue-summary-pipeline-1720217224


In [13]:
print(pipeline_experiment_name)

dialogue-summary-pipeline-1720217224


In [14]:
%store pipeline_experiment_name

Stored 'pipeline_experiment_name' (str)


# Create the `Trial`

In [15]:
from smexperiments.trial import Trial

In [16]:
%store -r pipeline_trial_name

timestamp = int(time.time())
pipeline_trial = Trial.create(
    trial_name="trial-{}".format(timestamp), experiment_name=pipeline_experiment_name, sagemaker_boto_client=sm
)
pipeline_trial_name = pipeline_trial.trial_name
print("Created Trial Name: {}".format(pipeline_trial_name))

no stored variable or alias pipeline_trial_name
Created Trial Name: trial-1720217224


In [17]:
print(pipeline_trial_name)

trial-1720217224


In [18]:
%store pipeline_trial_name

Stored 'pipeline_trial_name' (str)


# Define Parameters to Parametrize Pipeline Execution

We define Workflow Parameters by which we can parametrize our Pipeline and vary the values injected and used in Pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - representing a `str` Python type
* `ParameterInteger` - representing an `int` Python type
* `ParameterFloat` - representing a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

In [19]:
from sagemaker.workflow.parameters import (
    ParameterString,
    ParameterInteger,
    ParameterFloat
)

# Feature Engineering Step

In [20]:
%store -r raw_input_data_s3_uri

In [21]:
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-550703975190/data-summarization/


In [22]:
!aws s3 ls $raw_input_data_s3_uri

2024-07-05 21:48:46    6544107 dialogsum-1.csv
2024-07-05 21:48:46    6572423 dialogsum-2.csv


# Setup Pipeline Parameters
These parameters are used by the entire pipeline.

In [23]:
model_checkpoint='google/flan-t5-base'

In [24]:
model_checkpoint = ParameterString(
    name="ModelCheckpoint",
    default_value=model_checkpoint,
)

# Setup Processing Parameters

In [25]:
input_data = ParameterString(
    name="InputData",
    default_value=raw_input_data_s3_uri,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1,
)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.c5.xlarge",
)

train_split_percentage = ParameterFloat(
    name="TrainSplitPercentage",
    default_value=0.90,
)

validation_split_percentage = ParameterFloat(
    name="ValidationSplitPercentage",
    default_value=0.05,
)

test_split_percentage = ParameterFloat(
    name="TestSplitPercentage",
    default_value=0.05,
)

We create an instance of an `SKLearnProcessor` processor and we use that in our `ProcessingStep`.

In [26]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=432000,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


### _Ignore any `WARNING` ^^ above ^^._

### Setup Pipeline Step Caching
Cache pipeline steps for a duration of time using [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) format.  

More details on SageMaker Pipeline step caching here:  https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html

In [27]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="PT1H")

Finally, we use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` as the input data of the step itself. This input data will be used by the processor instance when it is run.

Also, take note the `"train"`, `"validation"` and `"test"` named channels specified in the output configuration for the processing job. Such step `Properties` can be used in subsequent steps and will resolve to their runtime values at execution. In particular, we'll call out this usage when we define our training step.

In [28]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processing_inputs = [
    ProcessingInput(
        input_name="raw-input-data",
        source=input_data,
        destination="/opt/ml/processing/input/data/",
        s3_data_distribution_type="ShardedByS3Key",
    )
]

processing_outputs = [
    ProcessingOutput(
        output_name="train",
        s3_upload_mode="EndOfJob",
        source="/opt/ml/processing/output/data/train",
    ),
    ProcessingOutput(
        output_name="validation",
        s3_upload_mode="EndOfJob",
        source="/opt/ml/processing/output/data/validation",
    ),
    ProcessingOutput(
        output_name="test",
        s3_upload_mode="EndOfJob",
        source="/opt/ml/processing/output/data/test",
    ),
]

processing_step = ProcessingStep(
    name="Processing",
    code="preprocess.py",
    processor=processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=[
        "--train-split-percentage",
        str(train_split_percentage.default_value),
        "--validation-split-percentage",
        str(validation_split_percentage.default_value),
        "--test-split-percentage",
        str(test_split_percentage.default_value),
        "--model-checkpoint",
        str(model_checkpoint.default_value),
    ],
    cache_config=cache_config
)

print(processing_step)

ProcessingStep(name='Processing', display_name=None, description=None, step_type=<StepTypeEnum.PROCESSING: 'Processing'>, depends_on=None)


# Train Step

# Setup Training Hyper-Parameters

In [29]:
train_instance_type = ParameterString(name="TrainInstanceType", default_value="ml.p2.xlarge")
train_instance_count = ParameterInteger(name="TrainInstanceCount", default_value=1)

In [30]:
epochs = ParameterInteger(name="Epochs", default_value=1)
learning_rate = ParameterFloat(name="LearningRate", default_value=0.00001)
weight_decay = ParameterFloat(name="WeightDecay", default_value=0.01)
train_batch_size = ParameterInteger(name="TrainBatchSize", default_value=1)
validation_batch_size = ParameterInteger(name="ValidationBatchSize", default_value=1)
test_batch_size = ParameterInteger(name="TestBatchSize", default_value=1)
train_volume_size = ParameterInteger(name="TrainVolumeSize", default_value=1024)
input_mode = ParameterString(name="InputMode", default_value="FastFile")
train_sample_percentage = ParameterFloat(name="TrainSamplePercentage", default_value=0.001)

### Setup Metrics To Track Model Performance

In [31]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "'train_loss': ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "'eval_loss': ([0-9\\.]+)"},
]

### Create the Estimator

We configure an Estimator and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later.

We also specify the model path where the models from training will be saved.

Note the `train_instance_type` parameter passed may be also used and passed into other places in the pipeline. In this case, the `train_instance_type` is passed into the estimator.

In [32]:
from sagemaker.pytorch import PyTorch
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

estimator = PyTorch(
    entry_point="train.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    py_version="py38",
    framework_version="1.12",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,        
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "model_checkpoint": model_checkpoint,
        "train_sample_percentage": train_sample_percentage,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions,
)

### Configure Training Step

Finally, we use the estimator instance to construct a `TrainingStep` as well as the `Properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to an estimator's `fit` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3Uri` of the `"train"`, `"validation"` and `"test"` output channel to the `TrainingStep`. The `properties` attribute of a Workflow step match the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved, or filled in, at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [33]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name="Train",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
        ),
        "validation": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
        ),
        "test": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
        ),
    },
    cache_config=cache_config,
)

print(training_step)

TrainingStep(name='Train', display_name=None, description=None, step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None)


/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/steps.py:445: UserWarning: Profiling is enabled on the provided estimator. The default profiler rule includes a timestamp which will change each time the pipeline is upserted, causing cache misses. If profiling is not needed, set disable_profiler to True on the estimator.
  warnings.warn(msg)


# Evaluation Step

First, we develop an evaluation script that will be specified in a Processing step that will perform the model evaluation.

The evaluation script `evaluate_model_metrics.py` takes the trained model and the test dataset as input, and produces a JSON file containing evaluation metrics.

After pipeline execution, we will examine the resulting `evaluation.json` for analysis.

The evaluation script:

* loads in the model
* reads in the test data
* issues a bunch of predictions against the test data
* builds an evaluation report
* saves the evaluation report to the evaluation directory

Next, we create an instance of a `SKLearnProcessor` and we use that in our `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [34]:
from sagemaker.sklearn.processing import SKLearnProcessor

evaluation_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=432000,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


### _Ignore any `WARNING` ^^ above ^^._

We use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [35]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(name="EvaluationReport", output_name="metrics", path="evaluation.json")

In [36]:
evaluation_step = ProcessingStep(
    name="EvaluateModel",
    processor=evaluation_processor,
    code="evaluate_model_metrics.py",
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/input/model"            
            
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/input/data"       
        ),
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output/metrics/",
            output_name="metrics", 
            s3_upload_mode="EndOfJob"            
        ),
    ],
    property_files=[evaluation_report],
)

In [37]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            evaluation_step.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

print(model_metrics)

# Register Model Step

![](img/pipeline-5.png)

We use the estimator instance that was used for the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a Model Package. A Model Package is a reusable model artifacts abstraction that packages all ingredients necessary for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A Model Package Group is a collection of Model Packages. You can create a Model Package Group for a specific ML business problem, and you can keep adding versions/model packages into it. Typically, we expect customers to create a ModelPackageGroup for a SageMaker Workflow Pipeline so that they can keep adding versions/model packages to the group for every Workflow Pipeline run.

The construction of `RegisterModel` is very similar to an estimator instance's `register` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Of note, we provided a specific model package group name which we will use in the Model Registry and CI/CD work later on.

In [38]:
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")

deploy_instance_type = ParameterString(name="DeployInstanceType", default_value="ml.m5.xlarge")

deploy_instance_count = ParameterInteger(name="DeployInstanceCount", default_value=2)

In [39]:
import time

timestamp = int(time.time())

model_package_group_name = f"Summarization-{timestamp}"

print(model_package_group_name)

Summarization-1720217228


In [40]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.12",
    instance_type=deploy_instance_type,
    image_scope="inference",
)
print(inference_image_uri)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py38


763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.12-cpu-py38


In [41]:
from sagemaker.workflow.step_collections import RegisterModel

register_step = RegisterModel(
    name="Summarization",
    estimator=estimator,
    image_uri=inference_image_uri,  # we have to specify, by default it's using training image
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/jsonlines"],
    response_types=["application/jsonlines"],
    inference_instances=[deploy_instance_type],
    transform_instances=[deploy_instance_type],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

# Create Model for Deployment Step

In [42]:
from sagemaker.model import Model

model_name = "model-{}".format(timestamp)

model = Model(
    name=model_name,
    image_uri=inference_image_uri,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sess,
    role=role,
)

In [43]:
from sagemaker.inputs import CreateModelInput

create_inputs = CreateModelInput(
    instance_type=deploy_instance_type,
)

In [44]:
from sagemaker.workflow.steps import CreateModelStep

create_step = CreateModelStep(
    name="CreateModel",
    model=model,
    inputs=create_inputs,
)

# Conditional Deployment Step
![](img/pipeline-6.png)

Finally, we'd like to only register this model if the metrics of the model, as determined by our evaluation step, exceeded a given threshold. A `ConditionStep` allows for pipelines to support conditional execution in the pipeline DAG based on conditions of step properties.

Below, we do the following:
* define a condition on the evaluation metrics found in the output of the evaluation step
* use the condition in the list of conditions in a `ConditionStep`
* pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`

In [45]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

min_rouge_value = ParameterFloat(name="MinRouge1Value", default_value=0.005)

min_rouge_condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report,
        json_path="metrics.eval_rouge1.value",
    ),
    right=min_rouge_value,  # eval_loss
)

min_rouge_condition_step = ConditionStep(
    name="EvaluationCondition",
    conditions=[min_rouge_condition],
    if_steps=[register_step, create_step],  # success, continue with model registration
    else_steps=[],  # fail, end the pipeline
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Define a Pipeline of Parameters, Steps, and Conditions

Let's tie it all up into a workflow pipeline so we can execute it, and even schedule it.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair so we tack on the timestamp to the name.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline need not be in the order of execution. The SageMaker Workflow service will resolve the _data dependency_ DAG as steps the execution complete.
* Steps must be unique to either pipeline step list or a single condition step if/else list.

# Submit the Pipeline to SageMaker for Execution 

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

# Create Pipeline

### _Ignore any `WARNING` below._

In [46]:
from sagemaker.workflow.pipeline import Pipeline

existing_pipelines = 0

existing_pipelines_response = sm.list_pipelines(
    PipelineNamePrefix=pipeline_name,
    SortOrder="Descending",
)

if "PipelineSummaries" in existing_pipelines_response.keys():
    if len(existing_pipelines_response["PipelineSummaries"]) > 0:
        existing_pipelines = existing_pipelines + 1
        print("[INFO] You already have created {} pipeline with name {}.".format(existing_pipelines, pipeline_name))
    else:
        pass

if existing_pipelines == 0:  # Only create the pipeline one time
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            input_data,
            processing_instance_count,
            processing_instance_type,
            train_split_percentage,
            validation_split_percentage,
            test_split_percentage,
            train_instance_type,
            train_instance_count,
            epochs,
            learning_rate,
            weight_decay,
            train_sample_percentage,
            train_batch_size,
            validation_batch_size,
            test_batch_size,
            train_volume_size,
            input_mode,
            min_rouge_value,
            model_approval_status,
            deploy_instance_type,
            deploy_instance_count,
            model_checkpoint.to_string(),
        ],
        steps=[processing_step, training_step, evaluation_step, min_rouge_condition_step],
        sagemaker_session=sess,
    )

    pipeline.upsert(role_arn=role)["PipelineArn"]
    print("Created pipeline with name {}".format(pipeline_name))
else:
    print(
        "****************************************************************************************************************"
    )
    print(
        "You have already create a pipeline with the name {}. This is OK. Please continue to the next cell.".format(
            pipeline_name
        )
    )
    print(
        "****************************************************************************************************************"
    )

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Created pipeline with name dialogue-summary-pipeline-1720217224


### _Ignore any `WARNING` ^^ above ^^._

# Start Pipeline

### _Ignore any `WARNING` below._

In [47]:
running_executions = 0
completed_executions = 0

if existing_pipelines > 0:
    existing_pipeline_executions_response = sm.list_pipeline_executions(
        PipelineName=pipeline_name,
        SortOrder="Descending",
    )

    if "PipelineExecutionSummaries" in existing_pipeline_executions_response.keys():
        if len(existing_pipeline_executions_response["PipelineExecutionSummaries"]) > 0:
            execution = existing_pipeline_executions_response["PipelineExecutionSummaries"][0]
            if "PipelineExecutionStatus" in execution:
                if execution["PipelineExecutionStatus"] == "Executing":
                    running_executions = running_executions + 1
                else:
                    completed_executions = completed_executions + 1

            print(
                "[INFO] You have {} Pipeline execution(s) currently running and {} execution(s) completed.".format(
                    running_executions, completed_executions
                )
            )
    else:
        pass
else:
    pass

if running_executions == 0:  # Only allow 1 pipeline execution at a time to limit the resources needed
    execution = pipeline.start()
    running_executions = running_executions + 1
    print("Started pipeline {}.  Ignore any warnings above.".format(pipeline_name))
    print(execution.arn)
else:
    print(
        "********************************************************************************************************************"
    )
    print(
        "You have already launched {} pipeline execution(s).  This is OK.  Please continue to see the next cell.".format(
            running_executions
        )
    )
    print(
        "********************************************************************************************************************"
    )

Started pipeline dialogue-summary-pipeline-1720217224.  Ignore any warnings above.
arn:aws:sagemaker:us-east-1:550703975190:pipeline/dialogue-summary-pipeline-1720217224/execution/bec0dga3fits


### _Ignore any `WARNING` ^^ above ^^._

# Wait for the Pipeline to Complete

### _This next cell takes about 40 mins.  Please be patient._

In [48]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Executing
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:550703975190:pipeline/dialogue-summary-pipeline-1720217224/execution/bec0dga3fits',
  'PipelineExecutionDisplayName': 'execution-1720217230882',
  'PipelineExecutionFailureReason': 'Step failure: One or multiple steps '
                                    'failed.',
  'PipelineExecutionStatus': 'Failed',
  'StartTime': datetime.datetime(2024, 7, 5, 22, 7, 10, 823000, tzinfo=tzlocal())}]
CPU times: user 6.22 s, sys: 282 ms, total: 6.5 s
Wall time: 5min 7s


### _Wait for the Pipeline ^^ Above ^^ to Complete_

# List Pipeline Execution Steps and Statuses After Completion

In [49]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]

print("Pipeline execution status {}".format(pipeline_execution_status))
print("Pipeline execution arn {}".format(pipeline_execution_arn))

Pipeline execution status Failed
Pipeline execution arn arn:aws:sagemaker:us-east-1:550703975190:pipeline/dialogue-summary-pipeline-1720217224/execution/bec0dga3fits


In [50]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 1,
                             'EndTime': datetime.datetime(2024, 7, 5, 22, 12, 17, 911000, tzinfo=tzlocal()),
                             'FailureReason': 'ClientError: Failed to invoke '
                                              'sagemaker:CreateTrainingJob. '
                                              'Error Details: The '
                                              'account-level service limit '
                                              "'ml.p2.xlarge for training job "
                                              "usage' is 1 Instances, with "
                                              'current utilization of 0 '
                                              'Instances and a request delta '
                                              'of 4 Instances. Please use AWS '
                                              'Service Quotas to request an '
                                              'increase for this quota. I

# List All Artifacts Generated by the Pipeline

In [51]:
processing_job_name = None
training_job_name = None

In [52]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

{'StepName': 'Processing', 'StartTime': datetime.datetime(2024, 7, 5, 22, 7, 11, 554000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 7, 5, 22, 12, 16, 311000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:550703975190:processing-job/pipelines-bec0dga3fits-Processing-mWb22q7UnX'}}, 'AttemptCount': 1}
pipelines-bec0dga3fits-Processing-mWb22q7UnX


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...864640ba832fa6b/input/code/preprocess.py,Input,DataSet,ContributedTo,artifact
1,s3://...-east-1-550703975190/data-summarization/,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...7224/bec0dga3fits/Processing/output/test,Output,DataSet,Produced,artifact
4,s3://...ec0dga3fits/Processing/output/validation,Output,DataSet,Produced,artifact
5,s3://...224/bec0dga3fits/Processing/output/train,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2024, 7, 5, 22, 12, 17, 37000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 7, 5, 22, 12, 17, 911000, tzinfo=tzlocal()), 'StepStatus': 'Failed', 'FailureReason': "ClientError: Failed to invoke sagemaker:CreateTrainingJob. Error Details: The account-level service limit 'ml.p2.xlarge for training job usage' is 1 Instances, with current utilization of 0 Instances and a request delta of 4 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.\nRetry not appropriate on execution of step with PipelineExecutionArn arn:aws:sagemaker:us-east-1:550703975190:pipeline/dialogue-summary-pipeline-1720217224/execution/bec0dga3fits and StepId Train. No retry policy configured for the exception type SAGEMAKER_RESOURCE_LIMIT.", 'Metadata': {}, 'AttemptCount': 1}


KeyError: 'TrainingJob'

## Add Execution Run as Trial to Experiments

In [ ]:
# -aws-processing-job is the default name assigned by ProcessingJob
processing_job_tc = "{}-aws-processing-job".format(processing_job_name)
print(processing_job_tc)

In [ ]:
response = sm.associate_trial_component(TrialComponentName=processing_job_tc, TrialName=pipeline_trial_name)

In [ ]:
# -aws-training-job is the default name assigned by TrainingJob
training_job_tc = "{}-aws-training-job".format(training_job_name)
print(training_job_tc)

In [ ]:
response = sm.associate_trial_component(TrialComponentName=training_job_tc, TrialName=pipeline_trial_name)